# Brightway2 seminar
Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Day 1, morning

### Learning objectives  
  - Learn about the general structure of Brightway and its most importand abstractions: projects, databases, activities and exchanges  
  - Learn how to find objects (notably activities and exchanges), assign them to variables and work with them using their associated methods  
  - Learn about how simple LCA calculations are done (one product, one impact category), and specifically how the different matrices are built and used  
  - Learn how to extract information from the matrices (inputs or results) and translate them into nice, human-readable objects  
  - Learn different ways to carry out comparative LCAs  
  - Learn different ways to carry out LCAs with multiple impact categories

### Content  

#### 1) Getting started  
##### 1.1) Downloading, installing and accessing Brightway2  
  - Downloading and installing Brightway  
  - Accessing Brightway2 libraries  
  
##### 1.2) Projects  
  - Concept  
  - Creating a new project, or switching to an existing project  
  - Contents of a project  

  
##### 1.3) bw2_setup(), biosphere3 database and LCIA methods  
  - bw2_setup()  
  - biosphere3 database and a first look at database objects  
  - Getting activities from codes or keys  
  - Methods  
  - Looking up elementary flows (list comprehensions, search)  
  - Searching for methods  
  - Nice display of data in methods 

##### 1.4) LCI databases  
  - Importing (succinct)  
  - LCI activities  
  - Looking up activities  
  - LCI exchanges
  - Loaded LCI databases
  
#### 2) My first LCA - simplest case:  
##### 2.1) General syntax of LCA calculations  

##### 2.2) The `demand` attribute  

##### 2.3) Reminder of the system that needs to be solved in calculating an LCI  

##### 2.4) Building the matrices  

  - $A$ matrix  
  - $B$ matrix  
  - $f$ (demand array)  
  
##### 2.5) Solution to the inventory calculation  

  - Supply array  
  - Inventory matrix  
  
##### 2.6) Life Cycle Impact Assessment  

  - `.lcia()` method  
  - Simple contribution analysis  
  
#### 3) My second LCA - comparative LCA:
    
#### 4) My third LCA - Multiple impact categories
  
#### 5) My first and third LCAs revisited with MultiLCA

### 1) Getting started

Like all other Python packages, you need to `import` Brightway2 modules. We will here import it as `bw`. This means that, to access a method or function from the brightway2 modules, the prefix `bw.` needs to be added. 

In [1]:
import brightway2 as bw

We're also going to be using the following libraries:

In [2]:
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

#### 1.2) Projects

##### Concept

The top-level containent in Brightway2 is the project (see [here](https://docs.brightwaylca.org/intro.html#projects) for a description and [here](https://docs.brightwaylca.org/technical/bw2data.html#projects) for the docs). A project contains LCI databases, LCIA methods and other less often used objects. Objects from one project do not interract with objects within other projects. By analogy, projects are like databases in openLCA and SimePro.  
When you first launch Brightway2, you will be in the `default` project. You can check this using the `current` property of the `projects` object: 

In [3]:
bw.projects.current

'default'

##### Creating a new project, or switching to an existing project

Let's create a new project for this seminar, unsurprisingly called "bw2_class_2020_intro". There are two ways of doing this:  
* `projects.create_project('bw2_class_2020_intro')` will create the project, but you will remain in your current project.
* `projects.set_current('bw2_class_2020_intro')` will switch you to the project passed as argument, and create it first if it doesn't exist.  Let's do the latter:

In [4]:
# The name of the project is entered as string; 
# it doesn't really have any restrictions, so can include spaces, 
# special characters, other languages, or even emoji.

bw.projects.set_current('bw2_class_2020_intro') 

You can always see what projects you have on your computer by running `list(bw.projects)`. Unless you have worked with Brightway2 before on your computer, your list should contain two projects: 'default' and 'bw2_seminar_2020'.

_**Exercise**_: list the projects on your computer.

In [6]:
list(bw.projects)

ValueError: unsupported pickle protocol: 5

Like in all Python modules, you can get additionnal information on the `projects` object and associated properties and methods by typing `help(projects)`. The [docs](https://docs.brightwaylca.org/technical/bw2data.html#projects) are of course more verbose.

##### Contents of a project

One property of `projects` is its location, given by `projects.dir`:

In [5]:
bw.projects.dir

'/Users/cmutel/Library/Application Support/Brightway3/bw2_class_2020_intro.c735de29b2ba138393428f80011c9425'

Looking at what is inside:  
<img src="images/project_folder_before_setup.JPG">

First things first: **do not panic**! You can use Brightway2 for years without ever opening this directory, but we will discuss some of these files later.

It is simply interesting to note that, for now, all the directories are empty except the `lci` directory, which contains an empty database.

All in all, the project takes up 4KB.  
Its now time to start populating the project.

#### 1.3) bw2setup(), biosphere3 database and LCIA methods

##### bw2setup()

The first thing you should do is add flow and LCIA methods. This is done by running the `bw2setup` function:

In [6]:
bw.bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/12/2020 21:16:16
  Finished: 03/12/2020 21:16:17
  Total time elapsed: 00:00:00
  CPU %: 86.50
  Memory %: 0.94
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.75 seconds
Wrote 850 LCIA methods with 219059 characterization factors
Creating core data migrations



The output tells us that bw2_setup created some very useful things:  
  - Created a database called "biosphere3": this database contains elementary flows (called biosphere exchanges in Brightway2)  
  - 718 impact assessment methods  
  
It also created some a `mapping` between the imported exchanges and some integer: more on this later.  
The whole directory now takes up 125MB.

In [7]:
bw.databases

Databases dictionary with 1 object(s):
	biosphere3

While not impossible to interact with the data at this level, you probably never will unless you are developping some funky program. Instead, it is strongly recommended to learn to work with `abstractions`.  

To access a database in Brightway, you use the `Database` initialization method (again, you can type `Database?` for more information - this is the last I'll mention this.

In [8]:
bw.Database('biosphere3')

Brightway2 SQLiteBackend: biosphere3

It doesn't actually return anything other than information about the Backend.  
However, there are many properties and functions associated with this database object.  These are found [here](https://docs.brightwaylca.org/technical/bw2data.html). We can also have a look through the autocomplete. Let's assign the database to a variable:

In [9]:
my_bio = bw.Database('biosphere3')

Let's check the my_bio `type`:

In [10]:
type(my_bio)

bw2data.backends.peewee.database.SQLiteBackend

Let's check its length:

In [11]:
len(my_bio)

4321

This is exactly the number of items we saw had been added to databases.db  
Given this, what do you think is going on?

If you type `my_bio.` and click on tab, you should get a list of properties and methods associated with database objects. Try this now:

In [12]:
my_bio.        #Type my_bio. and click tab. Have a look at the different properties and objects

SyntaxError: invalid syntax (<ipython-input-12-ad3d6eaab5a3>, line 1)

Some of the more basic ones we will be using now are :  
  - `random()` - returns a random activity in the database
  - `get(*valid_exchange_tuple*)` - returns an activity, but you must know the activity key
  - `load()` - loads the whole database as a dictionary.
  - `make_searchable` - allows searching of the database (by default, it is already searchable)
  - `search` - search the database  
  
Lets start with `random`:

In [13]:
my_bio.random()

'Dichlorvos' (kilogram, None, ('soil', 'agricultural'))

This returns a biosphere activity, but without assigning it to a variable, there is not much we can do with it directly.  

Note: It may seem counter-intuitive for elementary *flows* to be considered *activities* in Brightway, but it is no mistake. 
LCA models are made up of **nodes** (activities) that are linked by **edges** (exchanges). The biosphere activities are the nodes *outside* the technosphere. Emissions and resource extractions are modelled as exchanges between activities in the technosphere (part of the product system) and these biosphere activities.  

More on this later. 

For now, let's assign another random activity to a variable:

In [14]:
random_biosphere = my_bio.random()
random_biosphere

'Thulium, in ground' (kilogram, None, ('natural resource', 'in ground'))

We can get the type of the object that was returned from the database:

In [15]:
type(random_biosphere)

bw2data.backends.peewee.proxies.Activity

The type is an **activity proxy**. Activity proxies allow us to interact with the content of the database. In the journey to and from the database, several translation layers are used:  
<img src="images/data_transition_layers.png">

In Brightway, we *almost* always work with `Activity` or `Exchange` objects. 

To see what the activity contains, we can convert it to a dictionary:

In [16]:
random_biosphere.as_dict()

{'categories': ('natural resource', 'in ground'),
 'code': 'a34e010c-6f68-5c79-8b45-2955b91f7dc2',
 'CAS number': '007440-30-4',
 'name': 'Thulium, in ground',
 'database': 'biosphere3',
 'unit': 'kilogram',
 'type': 'natural resource'}

##### Getting activities from codes or keys

We can see that the activities in the biosphere3 database have unique **codes**, which we can use with the `get` function:

In [17]:
my_bio.get(random_biosphere['code'])

'Thulium, in ground' (kilogram, None, ('natural resource', 'in ground'))

Activities can also be "gotten" via `get_activity`, but the argument is the activity **key**, consisting of a tuple with two elements: the database name, and the activity code.

**Exercise:** Use `bw.get_activity()` to retrieve the random biosphere activity. 

In [18]:
database_name = 'biosphere3'
code = random_biosphere['code']
random_biosphere_key = (database_name, code)
random_biosphere_key

('biosphere3', 'a34e010c-6f68-5c79-8b45-2955b91f7dc2')

In [19]:
bw.get_activity(random_biosphere_key)

'Thulium, in ground' (kilogram, None, ('natural resource', 'in ground'))

You can always find (or return) the key to an activity using the `.key` property.

In [20]:
random_biosphere.key

('biosphere3', 'a34e010c-6f68-5c79-8b45-2955b91f7dc2')

##### Searching for activities

Let's say we are looking for a specific elementary flow, we can use the `search` method of the database (see [here](https://docs.brightwaylca.org/technical/bw2data.html#default-backend-databases-stored-in-a-sqlite-database) for more details on using search):

In [21]:
bw.Database('biosphere3').search('carbon dioxide')

['Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, from soil or biomass stock' (kilogram, None, ('air', 'low popula

It is also possible to use "filters" to narrow searches, e.g.

In [22]:
bw.Database('biosphere3').search('carbon dioxide', filter={'categories':'urban', 'name':'fossil'})

Excluding 18 filtered results


['Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))]

The database object is also iterable, allowing "home-made" searches through list comprehensions. This approach is better because one can add as many criteria as wanted:

In [23]:
[act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
         ]

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))]

Activities returned by searches or list comprehensions can be assigned to variables, but to do so, one needs to identify the activity by index. Based on the above, I can refine my filters to ensure the list comprehension only returns one activity, and then choose it without fear of choosing the wrong one..

In [24]:
[act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])]

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))]

In [25]:
activity_I_want = [act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
         ][0]
activity_I_want

'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))

**Exercise**: look for and assign to a variable an emission of nitrous oxide emitted to air in the "urban air" subcompartment.

In [26]:
# First inquiry:
[act for act in my_bio if 'nitrogen' in act['name']
                       and 'urban air' in str(act['categories'])
         ]

['Dinitrogen monoxide' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Dinitrogen monoxide' (kilogram, None, ('air', 'urban air close to ground'))]

In [27]:
# Found what I need:
[act for act in my_bio if 'Dinitrogen monoxide' in act['name']
                       and 'urban air close to ground' in str(act['categories'])
         ][0]

'Dinitrogen monoxide' (kilogram, None, ('air', 'urban air close to ground'))

Let's leave the biosphere database here for now.

##### Methods

bw2_setup() also installed LCIA methods.

In [28]:
bw.methods

Methods dictionary with 850 objects, including:
	('CML 2001 (obsolete)', 'acidification potential', 'average European')
	('CML 2001 (obsolete)', 'acidification potential', 'generic')
	('CML 2001 (obsolete)', 'climate change', 'GWP 100a')
	('CML 2001 (obsolete)', 'climate change', 'GWP 20a')
	('CML 2001 (obsolete)', 'climate change', 'GWP 500a')
	('CML 2001 (obsolete)', 'climate change', 'lower limit of net GWP')
	('CML 2001 (obsolete)', 'climate change', 'upper limit of net GWP')
	('CML 2001 (obsolete)', 'eutrophication potential', 'average European')
	('CML 2001 (obsolete)', 'eutrophication potential', 'generic')
	('CML 2001 (obsolete)', 'freshwater aquatic ecotoxicity', 'FAETP 100a')
Use `list(this object)` to get the complete list.

One can load a random method:

In [29]:
bw.methods.random()

('ReCiPe Endpoint (E,A) w/o LT (obsolete)',
 'ecosystem quality w/o LT',
 'terrestrial ecotoxicity w/o LT')

In [30]:
type(bw.methods.random())

tuple

Here, the random method returns the tuple by which the method is identified. To get to an actual method, the following syntax is used:

In [31]:
bw.Method(bw.methods.random())

Brightway2 Method: EDIP2003 (obsolete): non-renewable resources: aluminium

Of course, a random method is probably not useful except to play around. To find an actual method, one can again use list comprehensions. Let's say I am interested in using the IPCC2013 100 years method:

In [32]:
[m for m in bw.methods if 'IPCC' in str(m) and ('2013') in str(m) and '100' in str(m)]

[('IPCC 2013', 'climate change', 'GTP 100a'),
 ('IPCC 2013', 'climate change', 'GWP 100a'),
 ('IPCC 2013 no LT', 'climate change', 'GTP 100a'),
 ('IPCC 2013 no LT', 'climate change', 'GWP 100a')]

I am interested in the second of these, and will assign it to a variable. I can will refine my search until there is one element in my list and then choose it by subscripting.

In [33]:
[m for m in bw.methods if 'IPCC' in m[0]
                        and ('2013') in str(m)
                        and 'GWP 100' in str(m)
                        and 'no LT' not in str(m)]

[('IPCC 2013', 'climate change', 'GWP 100a')]

In [34]:
# Good, now let's choose it:
ipcc2013 = [m for m in bw.methods if 'IPCC' in m[0]
                    and ('2013') in str(m)
                    and 'GWP 100' in str(m)
                    and 'no LT' not in str(m)][0]

Of course, if I know exactly the method I want, and I know the syntax, I can simply type it out: `('IPCC 2013', 'climate change', 'GWP 100a')`

In [35]:
type(ipcc2013)

tuple

In [36]:
ipcc_2013_method = bw.Method(ipcc2013)

In [37]:
type(ipcc_2013_method)

bw2data.method.Method

Again, there are a bunch of methods associated with a method object. You can access these by typing ipcc_2013_method. and clicking tab.  
For example, metadata:

In [38]:
ipcc_2013_method.name

('IPCC 2013', 'climate change', 'GWP 100a')

In [39]:
ipcc_2013_method.metadata

{'description': "IPCC characterisation factors for the direct global warming potential of air emissions in ecoinvent 3.2. See the ecoinvent report 'Implementation of IPCC impact assessment method 2007 and 2013 to ecoinvent database 3.2 (2015.11.30)' for more details. Doesn't include: indirect formation of dinitrogen monoxide from nitrogen emissions, radiative forcing due to emissions of NOx, water, sulphate, etc. in the lower stratosphere + upper troposphere, carbon climate feedbacks, and any emissions which have cooling effects (e.g. black carbon). All CO is assumed to convert completely to CO2. Biogenic CO2 uptake and biogenic CO2 emissions are not characterised, except for 'Carbon dioxide, from soil or biomass stock' (deforestation and land transformation).",
 'filename': 'LCIA_implementation_3.6.xlsx',
 'unit': 'kg CO2-Eq',
 'abbreviation': 'ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf',
 'num_cfs': 211}

In [40]:
ipcc_2013_method.metadata['unit']

'kg CO2-Eq'

Question: where is this metadata?

Let's use the `load` method to see what is in the object:

In [41]:
ipcc_2013_method.load()

[(('biosphere3', 'e259263c-d1f1-449f-bb9b-73c6d0a32a00'), 1.0),
 (('biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'), 1.0),
 (('biosphere3', 'aa7cac3a-3625-41d4-bc54-33e2cf11ec46'), 1.0),
 (('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), 1.0),
 (('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'), 1.0),
 (('biosphere3', '5188df39-2bb8-4826-b469-fa9f86bacc09'), 1.0),
 (('biosphere3', 'e1c597cc-14cb-4ebb-af07-7a93a5b77d34'), 1.0),
 (('biosphere3', '6d89125e-e9b7-4d7e-a1fc-ada45dbd8815'), 1.0),
 (('biosphere3', '78eb1859-abd9-44c6-9ce3-f3b5b33d619c'), 1.0),
 (('biosphere3', 'e4e9febc-07c1-403d-8d3a-6707bb4d96e6'), 1.0),
 (('biosphere3', 'e8787b5e-d927-446d-81a9-f56977bbfeb4'), 1.0),
 (('biosphere3', '259cf8d6-6ea8-4ccf-84b7-23c930a5b2b3'), -1.0),
 (('biosphere3', '8ae4d8bb-3e4b-4825-8325-94696d7a64fd'), -1.0),
 (('biosphere3', '60d424f7-d5a9-4549-9540-da06684bc3bb'), -1.0),
 (('biosphere3', '375bc95e-6596-4aa1-9716-80ff51b9da77'), -1.0),
 (('biosphere3', '49c594c6-b6a8-42a3

This contains tuples with (elementary flow, characterization factors).

##### Nice display of data in methods 

**Exercise:** Create a dictionary with keys = elementary flow names and values = characterization factors for the ILCD "respiratory effects, inorganics" method (including long-term emissions).  
Bonus (optional): Generate a Pandas Series with the resulting dictionary. 

In [42]:
# First exploration
[m for m in bw.methods if 'ILCD' in str(m) 
                        and 'inorganics' in str(m)]

[('ILCD 1.0.8 2016 midpoint (obsolete)',
  'human health',
  'respiratory effects, inorganics'),
 ('ILCD 1.0.8 2016 midpoint no LT (obsolete)',
  'human health',
  'respiratory effects, inorganics'),
 ('ILCD 2.0 2018 midpoint', 'human health', 'respiratory effects, inorganics'),
 ('ILCD 2.0 2018 midpoint no LT',
  'human health',
  'respiratory effects, inorganics')]

In [43]:
# Refine search and assign to a variable
ILCD_resp_effects_tuple = [m for m in bw.methods if 'ILCD' in str(m) 
                        and 'inorganics' in str(m)
                        and 'no LT' not in str(m)][0]
ILCD_resp_effects = bw.Method(ILCD_resp_effects_tuple)
ILCD_resp_effects

Brightway2 Method: ILCD 1.0.8 2016 midpoint (obsolete): human health: respiratory effects, inorganics

In [44]:
# Generate the dictionary using a comprehension:
ILCD_resp_effects_dict = {bw.get_activity(ef[0])['name']:ef[1] for ef in ILCD_resp_effects.load()}
ILCD_resp_effects_dict

{'Ammonia': 0.0667,
 'Carbon monoxide, fossil': 0.000517,
 'Carbon monoxide, non-fossil': 0.000517,
 'Nitrogen oxides': 0.00778,
 'Particulates, < 2.5 um': 3.33,
 'Sulfur dioxide': 0.0667,
 'Sulfur trioxide': 0.0533}

In [45]:
# Bonus: put the whole thing in a neat Pandas series
pd.Series(ILCD_resp_effects_dict,
          name="{}, {}".format(ILCD_resp_effects.name, ILCD_resp_effects.metadata['unit']))

Ammonia                        0.066700
Carbon monoxide, fossil        0.000517
Carbon monoxide, non-fossil    0.000517
Nitrogen oxides                0.007780
Particulates, < 2.5 um         3.330000
Sulfur dioxide                 0.066700
Sulfur trioxide                0.053300
Name: ('ILCD 1.0.8 2016 midpoint (obsolete)', 'human health', 'respiratory effects, inorganics'), kg PM2.5-., dtype: float64

Enough said for now about methods.

#### 1.4) LCI datases

There is much information on the structure of LCI databases in Brightway2 [here](https://docs.brightwaylca.org/intro.html#inventory-databases), [here](http://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/Databases.ipynb) and [here](https://docs.brightwaylca.org/technical/bw2data.html#databases).  Probably the easiest way to learn about them, however, is to import one and have a look.  

We are going to import an example database to see how Brightway works. We'll start a new project to make sure we have a clear difference between what we did before, and what we will do now.

In [3]:
bw.projects.set_current('bw2_class_2020_example_database')

In [11]:
bw.add_example_database()

Writing activities to SQLite3 database:


Extracted 4 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: assign_only_product_as_production
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 11 strategies in 0.00 seconds
Applying strategy: link_iterable_by_fields


0% [#########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/13/2020 00:38:06
  Finished: 03/13/2020 00:38:06
  Total time elapsed: 00:00:00
  CPU %: 62.30
  Memory %: 0.76
Created database: Mobility example


Next, execute the following code. It will import the ecoinvent database. We will look in detail in another Notebook at what this code actually does.

In [12]:
bw.databases

Databases dictionary with 1 object(s):
	Mobility example

To access the actual database, you need to use the Database method: 

In [13]:
me = bw.Database('Mobility example')

This is a more advanced topic, but note that there are alternative backends. See [here](https://docs.brightwaylca.org/technical/bw2data.html#inventory-data-backends).

Let's assign the database to a variable and see what we can do:

In [14]:
# Check the length of the database:
len(me)

9

Again, we can get an idea of useful methods and attributes by typing eidb. and Tab. Do this now.

In [ ]:
me. #Press tab!

##### LCI activities

In the context of LCI databases, activities are the nodes "within the technosphere". They are therefore the columns in the technosphere matrix $A$.  
There are different ways to get access to an activity. Let's use the `random()` method for now to explore a random activity in the ecoinvent database.

In [15]:
random_act = me.random()

In [16]:
random_act

'Driving an combustion car' (kilometer, Europe, None)

In [17]:
type(random_act)

bw2data.backends.peewee.proxies.Activity

To see what is stored in an activity object, let's convert our random act in a dictionary: 

In [18]:
random_act.as_dict()

{'code': 'Driving an combustion car',
 'location': 'Europe',
 'unit': 'kilometer',
 'name': 'Driving an combustion car',
 'worksheet name': 'Gasoline car',
 'database': 'Mobility example',
 'reference product': 'Driving an combustion car',
 'production amount': 1.0}

Notice one important thing: **no exchanges**!  
Indeed, the exchanges and the activities are stored in two different tables of the `databases.db` database.  
It is possible, however, to iterate through the exchanges of the activities.

##### Searching and getting LCI activities

Searching and getting LCI activities is done exactly the same way as for activities in the biosphere3 database:

In [19]:
random_act.as_dict()

{'code': 'Driving an combustion car',
 'location': 'Europe',
 'unit': 'kilometer',
 'name': 'Driving an combustion car',
 'worksheet name': 'Gasoline car',
 'database': 'Mobility example',
 'reference product': 'Driving an combustion car',
 'production amount': 1.0}

In [20]:
# Using search
me.search('driv*')

['Driving an electric car' (kilometer, Europe, None),
 'Driving an combustion car' (kilometer, Europe, None)]

In [21]:
random_act['location']

'Europe'

In [ ]:
# Using list comprehensions:
[act for act in eidb if 'lorry' in act['name']
                    and 'RER' in act['location']
                    and '>32t' in act['name']
                    and 'operation' not in act['name']
]

**Exercise:** Return an activity for electricity production, coal-fired power plants in Germany

In [ ]:
[act for act in eidb if 'electricity' in act['name']
                        and 'coal' in act['name']
                        and act['location']=='DE'
              ][0]

#### LCI exchanges

**`Exchanges`** are the edges between nodes.

These can be:  
  - an edge between two activities within the technosphere (an element $a_{ij}$ of matrix $A$)  
  - edges between an activity in the technosphere and an activity in the "biosphere" (an element $b_{kj}$ of the biosphere matrix $B$).

One can iterate through **all** exchanges that have a given activity as `output`

In [22]:
for exc in random_act.exchanges():
    print(exc)

Exchange: 1.0 kilometer 'Driving an combustion car' (kilometer, Europe, None) to 'Driving an combustion car' (kilometer, Europe, None)>
Exchange: 5e-06 unit 'Combustion car' (unit, GLO, ('Combustion car',)) to 'Driving an combustion car' (kilometer, Europe, None)>
Exchange: 0.14259999999999998 kilogram 'CO2' (kilogram, None, None) to 'Driving an combustion car' (kilometer, Europe, None)>


One can also iterate through subsets of the exchanges:  
  - Technosphere exchanges: exchanges linking to other activities in the technosphere, `activity.technosphere()`  
  - Biosphere exchanges: AKA elementary flows, linking to activities in the biosphere database `activity.biosphere()`  
  - Production exchange: the reference flow of the activity `activity.production`  

Let's assign a **technosphere exchange** to a variable to learn more about it:

In [ ]:
random_techno_exchange = [exc for exc in random_act.technosphere()][0]
random_techno_exchange

In [ ]:
type(random_techno_exchange)

Again, the type is a proxy (refer to the diagram above about the different translation layers).

In [ ]:
# Amount, or weight of the edge
random_techno_exchange.amount

In [ ]:
# Activity the exchange stems from
random_techno_exchange.input

In [ ]:
# Activity the exchange terminates in
random_techno_exchange.output

In [ ]:
# Exchange as a dictionary
random_techno_exchange.as_dict()

Let's now look at a production exchange

**Exercise:** Assign a biosphere flow to a variable, and check the following:  
  - Is the output the same as for the technosphere exchange?  
  - From what database does the biosphere exchange come from?  
  - What is the amount of the exchange (i.e. the weight of the edge connecting the two activities)?
  
NOTE: If you get a ` list index out of range` error when trying to subscript your list comprehension, it means your list comprehension is empty, i.e. that there are no biosphere flows associated with the activity.

In [ ]:
# Assign the exchange to a variable:
random_bio_exchange = [exc for exc in random_act.biosphere()][0]
random_bio_exchange

In [ ]:
# Output of biosphere exchange
random_bio_exchange.output

In [ ]:
# Is it the same as the output of the technosphere exchange? It should be!
random_bio_exchange.output == random_techno_exchange.output

In [ ]:
# Database of the random biosphere exchange input - `.input`directly returns the activity proxy!
random_bio_exchange.input.key[0]

In [ ]:
# Amount of exchange
random_bio_exchange['amount']

## 2) My first LCA

Brightway has a so-called LCA object.  
It is instantiated using `LCA(args)`.  
The only required argument is a functional unit, described by a dictionary with keys = activities and values = amounts (more [here](https://docs.brightwaylca.org/lca.html#specifying-a-functional-unit)).  
A second argument that is often passed is an LCIA method, passed using the method tuple.  

### 2.1) General syntax of LCA calculations

Let's create our first LCA object using our random activity and our IPCC method.  

In [25]:
myFirstLCA_quick = bw.LCA({("Mobility example", "Driving an electric car"): 1}, ("IPCC", "simple"))

The steps to get to the impact score are as follows:

In [26]:
myFirstLCA_quick.lci()    # Builds matrices, solves the system, generates an LCI matrix.
myFirstLCA_quick.lcia()   # Characterization, i.e. the multiplication of the elements 
                          # of the LCI matrix with characterization factors from the chosen method
myFirstLCA_quick.score    # Returns the score, i.e. the sum of the characterized inventory

0.18248161259647167

Let's not take a closer look at the LCA object and its methods/attributes. We'll do this by creating a new LCA object: 

In [ ]:
myFirstLCA = bw.LCA({("Mobility example", "Driving an electric car"): 1}, ("IPCC", "simple"))

### 2.2) the `demand` attribute

In [ ]:
myFirstLCA.demand

To access the actual activity from the demand, you would do this:

In [ ]:
demanded_act = list(myFirstLCA.demand.keys())[0]
demanded_act

In [ ]:
demanded_act == random_act

There are also other attributes that have simply not been built yet, such as the `demand_array` and the `score`. To generate them, we first need to actually build the matrices. This will be done when calling the `.lci()` method.

### 2.3) Reminder of the system that needs to be solved in calculating an LCI

Before actually running the `.lci()` method, here's a quick refresher of the actual calculation that Brightway will need to do to calculate the inventory:  

$g=BA^{-1}f$  

where:  

  - $A$ = the technosphere matrix  
  - $B$ = the biosphere matrix (matrix with elementary flows)  
  - $f$ = the final demand vector  
  - $g$ = the inventory  

**Discussion:** Knowing what you do about the structure of Brightway (notably, activities and exchanges), what needs to happen to generate these matrices?

To consider:  
  - how should the order of the rows and columns be determined?  
  - how should we keep track of what is in each row and column?  
  - The parameters in the matrices are sometimes actually probability distribution functions - how should we consider this uncertainty information?  
  - The matrices are *sparse*, i.e. they are mostly made up of zeros. Should we consider this? Why? How?

### 2.4) Building the matrices

#### Structured arrays

LCI data imported in Brightway is stored in the `databases.db` database, discussed above.  
It is also stored in [numpy *structured arrays*](https://docs.scipy.org/doc/numpy/user/basics.rec.html). Here is a look at the  structured array for my ecoinvent 2.2 import (put in a pandas DataFrame because it looks nicer):  
<img src="images\structured_array_ecoinvent22.JPG">

**Exercise**(not core): Load the structured array of the ecoinvent database you are working with now.

In [ ]:
eidb.filepath_processed()

In [ ]:
your_structured_array = np.load(eidb.filepath_processed())
pd.DataFrame(your_structured_array).head()

In this array:  
  - `input` and `output` columns are integers that map to an activity. This mapping is found in the `mapping.pickle` file in the `project` directory and it looks something like this:

In [ ]:
pd.Series(bw.mapping).head()

  - `row` and `col`store *dummy* placeholder information about the location of the parameter in the matrices. 
  - the `type` indicates whether the exchange is a reference flow (`type`=0), technosphere exchange (`type`=1) or elementary flow (`type`=2).  
  - the other columns deal with uncertainty data. We'll cover that later, but one can always read about these columns in the [`stats_arrays` documentation](http://stats-arrays.readthedocs.io/en/latest/)

When the `.lci()` method is called, the structured arrayas are used to build matrices. The code responsible to do this is in the [`MatrixBuilder` class methods](https://bitbucket.org/cmutel/brightway2-calc/src/105e24e2d803c96773651ed73c43d850f9c23548/bw2calc/matrices.py?at=default&fileviewer=file-view-default). 

The method `MatrixBuilder.build_dictionary` is used to take input and output values, respectively, and figure out which rows and columns they correspond to. The actual code is succinct - only one line - but what it does is:
  - Get all unique values, as each value will appear multiple times
  - Sort these values
  - Give them integer indices, starting with zero
This information on row and column indices is sufficient to build matrices. These matrices are build in a [COOrdinate sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) format, where, for each exchange, three values are required: row position, column position, and amount (the actual value). The sparse matrices are actually stored in [CSR format](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix), but this is a detail.  

Some more details are are found [here](https://docs.brightwaylca.org/lca.html#building-matrices).  

Let's now finally run the `.lci()`.

In [ ]:
myFirstLCA.lci()

Here's what the structured arrays *now* look like:  

In [ ]:
pd.DataFrame(myFirstLCA.bio_params).head(5) # Technosphere parameters are at myFirstLCA.tech_params

We see that the row and col numbers are no longer dummy variables, but that they actually have real matrix indices.

#### Dictionaries that map between incides and activities

One of the useful things that the `MatrixBuilder` produces are `dictionaries` that map row and column numbers to the keys of activities.  There are three such dictionaries, all directly accessible as attributes of the LCA object:
 - `activity_dict`: Columns in the technosphere matrix $A$ or biosphere matrix $B$
 - `product_dict` : Rows in the technosphere matrix $A$  
 - `biosphere_dict`: Rows in the biosphere matrix $B$

Here what this dictionary looks like:

In [ ]:
myFirstLCA.activity_dict

So, if I know the key to my activity (which, again,  is a `tuple` consisting of the database name and the activity code), I can read the column index (from `activity_dict`) or row index (from `product_dict` or `biosphere_dict` for the $A$ or $B$ matrices, respectively). 

Let's find out what column is associated with the activity that is producing our final demand as reference flow.

In [ ]:
# Getting the key from the `demand`attribute:
act_key = list(myFirstLCA.demand)[0].key
# Getting the column number from the activity_dict:
col_index = myFirstLCA.activity_dict[act_key]
print("The column index for activity {} is {}".format(act_key, col_index))

While this is useful, it is often more useful to determine what a row or column in the matrices actually refers to. In these cases, we need a dictionary that maps row or column indices to activity keys, and not the opposite.  
We can do this by reversing our dictionaries:

In [ ]:
myFirstLCA_rev_activity_dict = {value:key for key, value in myFirstLCA.activity_dict.items()}
myFirstLCA_rev_activity_dict

As a convenience, Brightway offers a method that will generate the three reverse dictionaries simultaneously.  
`.reverse_dict()` returns three reverse dicts (reverse activity dict,  reverse product dict,  reverse biosphere dict) *in that order*. The syntax for creating and assigning these reverse dicts is therefore: 

In [ ]:
myFirstLCA_rev_act_dict, myFirstLCA_rev_product_dict, myFirstLCA_rev_bio_dict = myFirstLCA.reverse_dict()

#### $A$ and $B$ Matrices

We can also access the matrices that were constructed. Let's look at the technosphere matrix ($A$).  
The ** $A$ matrix**, with each element $a_{ij}$ provides information on the amount of input or output of product $i$ from activity $j$. When $i=j$, the element $a_{ij}$ is the *reference flow* for the activity described in the column.

In [ ]:
myFirstLCA.technosphere_matrix

I am told that the dimensions of the matrix is $n*n$ where $n$ is the number of activities in my product system, and that the amount of actually stored elements is much less than $n^2$ (because the matrix is *sparse* and zero values are not stored).  

We can have an idea of what it stores by printing it out:

In [ ]:
print(myFirstLCA.technosphere_matrix)

It therefore stores both the coordinates and the values (as expected).
We can slice this matrix using coordinates. For example, let's say we wanted a view of the exchanges associated with the unit process providing our functional unit.  
We already know found the column number for that activity: 

In [ ]:
print("As a reminder, the column index for  {} is  {}".format(act_key, col_index))

To return the whole column from the $A$ matrix, we therefore slice the $A$ matrix.  
Python notes:  
  - In Python, slicing is done using []
  - We specify rows first, then columns  
  - `:` refers to "the whole row" or "the whole column" (depending if it is passed first or second in the []) 

In [ ]:
myColumn = myFirstLCA.technosphere_matrix[:, col_index]
myColumn

Printing this out gives:

In [ ]:
print(myColumn)

Not too useful: it would be better to get the *names* to these exchanges.  
We need to do two things:  
  - Get the indices from the CSR matrix (we can do this by converting it to a sparse matrix in `COOrdinate` format first)  
  - Get the activity code for the each index (we can do this using the reverse of the `activity_dict`)  
  - Use `get_activity` to access the actual names of the activities.  

1) Converting the CSR matrix to a COO matrix:  

In [ ]:
myColumnCOO = myColumn.tocoo()
myColumnCOO

It is still a sparse matrix with the same number of elements, and it looks quite like the CSR version when we print it out:

In [ ]:
print(myColumnCOO)

However, we can directly access the rows and column indices using `row` and `col`:  

In [ ]:
myColumnCOO.row

2) Get the activity code for each element using the **reverse** product dictionary we produced above:

In [ ]:
# Using a list comprehension:
[myFirstLCA_rev_product_dict[i] for i in myColumnCOO.row]

It would be even nicer to get the names for these:

In [ ]:
names_of_my_inputs = [bw.get_activity(myFirstLCA_rev_product_dict[i])['name'] for i in myColumnCOO.row]
names_of_my_inputs

We can put these in a neat Pandas Series, with actual names and amounts:

In [ ]:
# First create a dict with the information I want:
myColumnAsDict = dict(zip(names_of_my_inputs,myColumnCOO.data))
# Create Pandas Series from dict
pd.Series(myColumnAsDict, name="Nice series with information on exchanges in my foreground process")

Alternative way to generate similar information without even looking at the matrices:

In [ ]:
pd.Series({bw.get_activity(exc.input)['name']:exc.amount for exc in random_act.technosphere()}, 
          name="alternative way to generate exchanges")

Note the differences:  
  - The reference flow is not there (activity.technosphere() only returns technoshere exchanges where the input is not equal to the output)  
  - The values are positive, not negative (because the $A$ matrix is $I-Z$ where $Z$ contains the information on these inputs.

**Exercise**: Create a Pandas Series with the elementary flows of the activity supplying the reference flow for myFirstLCA.

In [ ]:
# Solution
myBioColumn = myFirstLCA.biosphere_matrix[:, col_index]
myBioValues = myBioColumn.tocoo().data
myBioNames = [bw.get_activity(myFirstLCA_rev_bio_dict[row])['name'] for row in myBioColumn.tocoo().row]
pd.Series(dict(zip(myBioNames, myBioValues)))

#### Demand array $f$

The demand array is the $f$ in $As=f$. 
It is an attribute of the LCA object:

In [ ]:
myFirstLCA.demand_array

Looks like it is all zeros, but not so:

In [ ]:
myFirstLCA.demand_array.sum()

So where is the one? We can know this by using our `activity_dict`.

In [ ]:
demand_database = list(myFirstLCA.demand.keys())[0]['database']
demand_code = list(myFirstLCA.demand.keys())[0]['code']
(demand_database, demand_code)

In [ ]:
row_of_demand = myFirstLCA.activity_dict[(demand_database, demand_code)]
row_of_demand # Row number of our demand vector containing the functional unit.

In [ ]:
myFirstLCA.demand_array[row_of_demand]

### 2.5) Solution to the inventory calculation

We saw above how `.lci()` produced the $A$ and $B$ matrices.  
`.lci()` also *solves* the equation $As=f$ and calculated the inventory by multiplying the solution to this equation by the biosphere matrix.  

#### Supply array

Vector containing the amount each activity will need to provide to meet the functional demand, i.e. $s=A^{-1}f$.

In [ ]:
myFirstLCA.supply_array

In [ ]:
myFirstLCA.supply_array.shape

**Inventory matrix**  
Contains the inventory *by activity* (i.e. not summed). In other words, we do not have $g=BA^{-1}f$, but rather $G=B \cdot diag(A^{-1}f)$

In [ ]:
myFirstLCA.inventory

We can aggregate the LCI results along the columns (i.e. calculate the cradle-to-gate inventory):

In [ ]:
LCI_cradle_to_gate = myFirstLCA.inventory.sum(axis=1)
LCI_cradle_to_gate.shape

**Exercise:** Get the total (cradle-to-gate) emissions of nitrous oxide emitted to air in the "urban air" subcompartment.

In [ ]:
NOx_act = [act for act in my_bio if 'Dinitrogen monoxide' in act['name']
                       and 'urban air close to ground' in str(act['categories'])
         ][0]
NOx_act.key

In [ ]:
NOx_row = myFirstLCA.biosphere_dict[NOx_act]
NOx_row

In [ ]:
myFirstLCA.inventory[NOx_row, :].sum()

### 2.7) LCIA

The LCIA calculation is done via the `.lcia()` method.

In [ ]:
myFirstLCA.lcia()

A number of other matrices are now available:

In [ ]:
# Matrix of characterization factors:
myFirstLCA.characterization_matrix

In [ ]:
myFirstLCA.characterization_matrix.shape

In [ ]:
# Matrix of characterized inventory flows
myFirstLCA.characterized_inventory

Question: would there be more, less or just as many elements in the inventory matrix as there are in the characterized inventory matrix?

The overall score is now an attribute of the LCA object: 

In [ ]:
myFirstLCA.score

We also could have determined what this score was by summing the elements of our `characterized_inventory` matrix:

In [ ]:
myFirstLCA.characterized_inventory.sum()

We could also have calculated it by multiplying the inventory and characterization factors ourselves:

In [ ]:
(myFirstLCA.characterization_matrix * myFirstLCA.inventory).sum()

We could also calculate the score by elementary flow (summing columns for each rows), irrespective of the unit process that produced it:

In [ ]:
elementary_flow_contribution = myFirstLCA.characterized_inventory.sum(axis=1) #Axis is the dimension I want to sum over:

In [ ]:
elementary_flow_contribution.shape

Notice that is has **two** dimensions. The result is in fact a one-dimensional matrix:

In [ ]:
type(elementary_flow_contribution)

To convert it to an array (probably more useful for many purposes), you can use any of the following approaches:

In [ ]:
elementary_flow_contribution.A1 
#np.squeeze(np.asarray(elementary_flow_contribution))
#np.asarray(elementary_flow_contribution).reshape(-1)
#np.array(elementary_flow_contribution).flatten()
#np.array(elementary_flow_contribution).ravel()

**Exercise:** Create a Pandas series that has the scores per unit process, sorted by value (contribution analysis)

In [ ]:
# Create array with the results per column (i.e. per activity)
results_by_activity = (myFirstLCA.characterized_inventory.sum(axis=0)).A1

In [ ]:
# Create a list of names in columns
list_of_names_in_columns = [bw.get_activity(myFirstLCA_rev_act_dict[col])['name'] 
                            for col in range(myFirstLCA.characterized_inventory.shape[1])]

In [ ]:
pd.Series(index=list_of_names_in_columns, data=results_by_activity).sort_values(ascending=False).head(10)

## 3) My second LCA: Comparative LCA

Let's choose two activities to compare, say Swiss electricity produced from respectively a run-of-river hydropower plant and a wind turbine.

**Exercise**: assign the two activities to variables `hydro` and `wind` respectively.

In [ ]:
[act for act in eidb if "wind" in act['name'] and "electricity" in act['name'] and "CH" in act['location']]

In [ ]:
wind = [act for act in eidb if "wind" in act['name']
        and "600kW" in act['name'] 
        and "kilowatt hour" in act['unit']][0]
wind

In [ ]:
[act for act in eidb if "hydro" in act['name'] and "river" in act['name'] and "CH" in act['location']]

In [ ]:
hydro = [act for act in eidb if "hydro" in act['name'] 
                     and "river" in act['name'] 
                     and "CH" in act['location']
                     and "kilowatt hour" in act['unit']
                     ][0]
hydro

Let's also compare these according to their carbon footprint as measured with the IPCC method we already selected above:

In [ ]:
ipcc_2013_method

#### One at a time approach:

In [ ]:
hydroLCA = bw.LCA({hydro:1}, ipcc_2013_method.name)
hydroLCA.lci()
hydroLCA.lcia()
hydroLCA.score

**Exercise:** Do the LCA for `wind`:

In [ ]:
windLCA = bw.LCA({wind:1}, ipcc_2013_method.name)
windLCA.lci()
windLCA.lcia()
windLCA.score

In [ ]:
#Compare results:
if windLCA.score>hydroLCA.score:
    print("Hydro is preferable")
elif windLCA.score<hydroLCA.score:
    print("Wind is preferable")
else:
    print("Both options have the same climate change indicator result")

Do one "delta" LCA:

In [ ]:
deltaLCA = bw.LCA({wind:1, hydro:-1}, ipcc_2013_method.name)
deltaLCA.lci()
deltaLCA.lcia()
deltaLCA.score

In [ ]:
if deltaLCA.score>0:
    print("Hydro is preferable")
elif deltaLCA.score<0:
    print("Wind is preferable")
else:
    print("Both options have the same climate change indicator result")

## 4) My third LCA - Multiple impact categories

Say we want to evaluate the indicator results for our randomAct for all ILCD midpoint categories (with long-term emissions).

In [ ]:
# Make a list of all impact method names (tuples):
ILCD = [method for method in bw.methods if "ILCD" in str(method) and "no LT" not in str(method)]
ILCD

Simplest way: for loop, using `switch method`

In [ ]:
myThirdLCA = bw.LCA({random_act:1}, ILCD[0]) # Do LCA with one impact category
myThirdLCA.lci()
myThirdLCA.lcia()
for category in ILCD:
    myThirdLCA.switch_method(category)
    myThirdLCA.lcia()
    print("Score is {:f} {} for category {}".format(myThirdLCA.score, 
                                                 bw.Method(category).metadata['unit'],
                                                 bw.Method(category).name)
          )

In [ ]:
myFirstLCA_unitProcessContribution = myFirstLCA.characterized_inventory.sum(axis=0).A1
myFirstLCA_unitProcessRelativeContribution = myFirstLCA_unitProcessContribution/myFirstLCA.score

## Revising my second and third LCA with `MultiLCA`

The `MultiLCA` allows thecalculation of LCA results for multiple functional units and impact categories.  
One simply needs to create a `calculation setup`, i.e. a named set of functional units and LCIA methods.

Calculation setups: dictionary with lists of functional units and methods.

In [ ]:
list_functional_units = [{wind.key:1}, {hydro.key:1}]
list_methods = ILCD

In [ ]:
bw.calculation_setups['wind_vs_hydro'] = {'inv':list_functional_units, 'ia':list_methods}

In [ ]:
bw.calculation_setups['wind_vs_hydro']

In [ ]:
myMultiLCA = bw.MultiLCA('wind_vs_hydro')

In [ ]:
myMultiLCA.results.shape

In [ ]:
myMultiLCA.results

In [ ]:
pd.DataFrame(index=ILCD, columns=[wind['name'], hydro['name']], data=myMultiLCA.results.T)

You can also create "fuller" DataFrames. Here is with code from [here](http://stackoverflow.com/questions/42984831/create-a-dataframe-from-multilca-results-in-brightway2): 

In [ ]:
scores = pd.DataFrame(myMultiLCA.results, columns=myMultiLCA.methods)
as_activities = [
    (bw.get_activity(key), amount) 
    for dct in myMultiLCA.func_units 
    for key, amount in dct.items()
]
nicer_fu = pd.DataFrame(
    [
        (x['database'], x['code'], x['name'], x['location'], x['unit'], y) 
        for x, y in as_activities
    ], 
    columns=('Database', 'Code', 'Name', 'Location', 'Unit', 'Amount')
)
pd.concat([nicer_fu, scores], axis=1).T

You can even generate beautiful heatmaps like this in a relatively easy way, see example notebook [here](http://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/Using%20calculation%20setups.ipynb).

<img src="images/multiLCA_heatmap.JPG">

Done with deterministic LCA using only existing database items!